# Lesson 2 Demo 1: Creating Normalized Tables

<img src="images/postgresSQLlogo.png" width="250" height="250">

## Walk through the basics of modeling data in normalized form. <br>
<ol><li>Create tables in PostgreSQL
<li>Insert rows of data
<li>Do a simple JOIN SQL query to show how these tables can work together.

### Import the library 
Note: An error might popup after this command has executed. If it does, read it carefully before ignoring. 

In [1]:
import psycopg2

### Create a connection to the database, get a cursor, and set autocommit to true

In [7]:
# Create a connection

try: 
    conn = psycopg2.connect("host=dbstream.cyq4sncu0rtz.us-east-1.rds.amazonaws.com dbname=music user=master password=master123")
    conn.set_session(autocommit=True)
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Could not get cursor to the database")
    print(e)

#### Let's imagine we have a table called Music Library. 

`Table Name: music_library
column 0: Album Id
column 1: Album Name
column 2: Artist Name
column 3: Year 
column 4: List of songs`

<img src="images/table1.png" width="650" height="650">



#### Now to translate this information into a Create Table Statement and insert the data

In [16]:
# Create a Table statement

try: 
    cur.execute("DROP TABLE music_library")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    
    
try: 
    cur.execute("CREATE TABLE if not exists music_library (album_id int, album_name varchar, artist_name varchar, year int, list_of_songs text[])")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    

# Include the Insert statement    
try: 
    cur.execute("INSERT INTO music_library (album_id, album_name, artist_name, year, list_of_songs) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (1, "Rubber Soul", "The Beatles", 1965, ["Michelle", "Think For Yourself", "In My Life"]))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_library (album_id, album_name, artist_name, year, list_of_songs) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (2, "Let It Be", "The Beatles", 1970, ["Let It Be", "Across The Universe"]))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
# Confirm the data got inserted in the table we created    
try: 
    cur.execute("SELECT * FROM music_library;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, ['Michelle', 'Think For Yourself', 'In My Life'])
(2, 'Let It Be', 'The Beatles', 1970, ['Let It Be', 'Across The Universe'])


#### Moving to 1st Normal Form (1NF)
This data has not been normalized. To get this data into 1st normal form, we will need to remove any collections or list of data. We need to break up the list of songs into individuals rows. 


`Table Name: music_library2
column 0: Album Id
column 1: Album Name
column 2: Artist Name
column 3: Year 
column 4: Song Name`

<img src="images/table2.png" width="650" height="650">

In [17]:
    
try: 
    cur.execute("CREATE TABLE if not exists music_library2 (album_id int, album_name varchar, artist_name varchar, year int, song varchar)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    

# Include the Insert statement    
try: 
    cur.execute("INSERT INTO music_library2 (album_id, album_name, artist_name, year, song) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (1, "Rubber Soul", "The Beatles", 1965, "Michelle"))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO music_library2 (album_id, album_name, artist_name, year, song) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (1, "Rubber Soul", "The Beatles", 1965, "Think For Yourself"))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_library2 (album_id, album_name, artist_name, year, song) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (1, "Rubber Soul", "The Beatles", 1965, "In My Life"))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_library2 (album_id, album_name, artist_name, year, song) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (2, "Let It Be", "The Beatles", 1970, "Let It Be"))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO music_library2 (album_id, album_name, artist_name, year, song) \
                 VALUES (%s, %s, %s, %s, %s)", \
                 (2, "Let It Be", "The Beatles", 1970, "Across The Universe"))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)



# Confirm the data got inserted in the table we created    
try: 
    cur.execute("SELECT * FROM music_library2;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, 'Michelle')
(1, 'Rubber Soul', 'The Beatles', 1965, 'Think For Yourself')
(1, 'Rubber Soul', 'The Beatles', 1965, 'In My Life')
(2, 'Let It Be', 'The Beatles', 1970, 'Let It Be')
(2, 'Let It Be', 'The Beatles', 1970, 'Across The Universe')


#### Moving to 2nd Normal Form (2NF)
We have moved our data to be in 1NF which is the first step in moving to 2nd Normal Form. Our table is not yet in 2nd Normal Form. While each of our records in our table is unique, our Primary key (*album id*) is not unique. We need to break this up into two tables, *album library* and *song library*. 

`Table Name: album_library 
column 0: Album Id
column 1: Album Name
column 2: Artist Name
column 3: Year `

`Table Name: song_library
column 0: Song Id
column 1: Song Name
column 3: Album Id` 

<img src="images/table3.png" width="450" height="450"> <img src="images/table4.png" width="450" height="450">

In [30]:

try: 
    cur.execute("DROP TABLE album_library ")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    
try: 
    cur.execute("DROP TABLE song_library")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    
    
try: 
    cur.execute("CREATE TABLE if not exists album_library (album_id int, album_name varchar, artist_name varchar, year int)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    

# Include the Insert statement    
try: 
    cur.execute("INSERT INTO album_library (album_id, album_name, artist_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (1, 'Rubber Soul', 'The Beatles', 1965))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO album_library (album_id, album_name, artist_name, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (2, 'Let It Be', 'The Beatles', 1970))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

    
    
try: 
    cur.execute("CREATE TABLE if not exists song_library (song_id int, song_name varchar, album_id int)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    
try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (1,'Michelle', 1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

    
try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (2,'Think For Yourself', 1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

    
try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (3,'In My Life', 1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)


try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (4,'Let It Be', 2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)


try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (5,'Across The Universe', 2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

    

try: 
    cur.execute("select * from album_library")
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

print("1st table")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    
try: 
    cur.execute("select * from song_library")
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

print("2nd table")

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

print("Table print done")
# Confirm the data got inserted in the table we created    


1st table
(1, 'Rubber Soul', 'The Beatles', 1965)
(2, 'Let It Be', 'The Beatles', 1970)
2nd table
(1, 'Michelle', 1)
(2, 'Think For Yourself', 1)
(3, 'In My Life', 1)
(4, 'Let It Be', 2)
(5, 'Across The Universe', 2)
Table print done


#### Let's do a `JOIN` on this table so we can get all the information we had in our first Table. 

In [31]:
try: 
    cur.execute("SELECT * FROM album_library JOIN song_library on album_library.album_id = song_library.album_id;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, 1, 'Michelle', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 2, 'Think For Yourself', 1)
(1, 'Rubber Soul', 'The Beatles', 1965, 3, 'In My Life', 1)
(2, 'Let It Be', 'The Beatles', 1970, 4, 'Let It Be', 2)
(2, 'Let It Be', 'The Beatles', 1970, 5, 'Across The Universe', 2)


#### Moving to 3rd Normal Form (3NF)
Check our table for any transitive dependencies. *Album_library* can move *Artist_name* to its own table, called *Artists*, which will leave us with 3 tables. 

`Table Name: album_library2 
column 0: Album Id
column 1: Album Name
column 2: Artist Id
column 3: Year `

`Table Name: song_library
column 0: Song Id
column 1: Song Name
column 3: Album Id`

`Table Name: artist_library
column 0: Artist Id
column 1: Artist Name `
<img src="images/table4.png" width="450" height="450"> <img src="images/table5.png" width="450" height="450"> <img src="images/table6.png" width="350" height="350">


In [32]:
try: 
    cur.execute("DROP TABLE album_library ")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    
try: 
    cur.execute("DROP TABLE song_library")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    
    
try: 
    cur.execute("CREATE TABLE if not exists album_library (album_id int, album_name varchar, artist_id int, year int)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    

# Include the Insert statement    
try: 
    cur.execute("INSERT INTO album_library (album_id, album_name, artist_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (1, 'Rubber Soul', 1, 1965))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("INSERT INTO album_library (album_id, album_name, artist_id, year) \
                 VALUES (%s, %s, %s, %s)", \
                 (2, 'Let It Be', 1, 1970))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

    
    
try: 
    cur.execute("CREATE TABLE if not exists song_library (song_id int, song_name varchar, album_id int)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
    
try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (1,'Michelle', 1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

    
try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (2,'Think For Yourself', 1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

    
try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (3,'In My Life', 1))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)


try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (4,'Let It Be', 2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)


try: 
    cur.execute("INSERT INTO song_library (song_id , song_name , album_id) \
                 VALUES (%s, %s, %s)", \
                 (5,'Across The Universe', 2))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

    

try: 
    cur.execute("CREATE TABLE if not exists artist_library (artist_id int, artist_name varchar)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    

try: 
    cur.execute("INSERT INTO artist_library (artist_id , artist_name ) \
                 VALUES (%s, %s)", \
                 (1,'The Beatles'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute("select * from album_library")
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

print("1st table")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    
try: 
    cur.execute("select * from song_library")
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

print("2nd table")

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    
try: 
    cur.execute("select * from artist_library")
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

print("3rd table")

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

print("Table print done")
# Confirm the data got inserted in the table we created    


1st table
(1, 'Rubber Soul', 1, 1965)
(2, 'Let It Be', 1, 1970)
2nd table
(1, 'Michelle', 1)
(2, 'Think For Yourself', 1)
(3, 'In My Life', 1)
(4, 'Let It Be', 2)
(5, 'Across The Universe', 2)
3rd table
(1, 'The Beatles')
Table print done


#### Let's do two `JOIN` on these 3 tables so we can get all the information we had in our first Table. 

In [33]:
try: 
    cur.execute("SELECT * FROM album_library JOIN song_library on album_library.album_id = song_library.album_id JOIN artist_library on album_library.artist_id = artist_library.artist_id;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 1, 1965, 1, 'Michelle', 1, 1, 'The Beatles')
(1, 'Rubber Soul', 1, 1965, 2, 'Think For Yourself', 1, 1, 'The Beatles')
(1, 'Rubber Soul', 1, 1965, 3, 'In My Life', 1, 1, 'The Beatles')
(2, 'Let It Be', 1, 1970, 4, 'Let It Be', 2, 1, 'The Beatles')
(2, 'Let It Be', 1, 1970, 5, 'Across The Universe', 2, 1, 'The Beatles')


### DONE! We have Normalized our dataset! 

### For the sake of the demo, I will drop the tables. 

### And finally close your cursor and connection. 

In [34]:
cur.close()
conn.close()